In [1]:
import pyspark_cassandra

In [2]:
cf_transaksi_standard = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_transaksi_standard", keyspace="skripsi").load()
cf_buku = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_buku", keyspace="skripsi").load()
cf_transaksi_jurusan_kategori = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_transaksi_jurusan_kategori", keyspace="skripsi").load()
mv_transaksi_kondisi_koleksi = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="mv_transaksi_kondisi_koleksi", keyspace="skripsi").load()
cf_usulan = sqlContext.read.format('org.apache.spark.sql.cassandra').\
                options(table="cf_usulan", keyspace="skripsi").load()

In [3]:
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("select count(*) from cf_transaksi_standard").show()

+--------+
|count(1)|
+--------+
|  501704|
+--------+



In [4]:
#query 1
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("""WITH cte as (
                    SELECT kode_anggota,
                           nama_jurusan,
                           fakultas,
                           tahun_ajaran, 
                           COUNT(unique_id) AS jumlah,
                           ROW_NUMBER() OVER (PARTITION BY tahun_ajaran ORDER BY COUNT(unique_id) DESC) AS r 
                    FROM cf_transaksi_standard
                    WHERE tahun_ajaran IN ('2014/2015','2015/2016','2016/2017')
                    GROUP BY kode_anggota,nama_jurusan,fakultas,tahun_ajaran
                )
                SELECT kode_anggota,nama_jurusan,fakultas,tahun_ajaran,jumlah FROM cte where r<=3 
                ORDER BY tahun_ajaran,jumlah""").show()

+------------+--------------+--------+------------+------+
|kode_anggota|  nama_jurusan|fakultas|tahun_ajaran|jumlah|
+------------+--------------+--------+------------+------+
|      94-029|      karyawan|karyawan|   2014/2015|   650|
|    11412046|Sastra Inggris|  Sastra|   2014/2015|   678|
|     MP15030|      karyawan|karyawan|   2014/2015|   706|
|       46121|      karyawan|karyawan|   2015/2016|   594|
|     MP15030|      karyawan|karyawan|   2015/2016|   780|
|      00-007|      karyawan|karyawan|   2015/2016|   920|
|      03-030|      karyawan|karyawan|   2016/2017|   256|
|      86-002|      karyawan|karyawan|   2016/2017|   284|
|      98-011|      karyawan|karyawan|   2016/2017|   324|
+------------+--------------+--------+------------+------+



In [ ]:
#query 2
cf_transaksi_standard.registerTempTable("cf_transaksi_standard")
sqlContext.sql("""SELECT YEAR(tanggal_pinjam) AS tahun,
                         MONTH(tanggal_pinjam) AS Bulan, 
                         SUM(denda) AS Jumlah, 
                         SUM(denda-terbayar) AS Belum_Terbayar
                    FROM cf_transaksi_standard
                    WHERE tahun_ajaran='2015/2016'
                    GROUP BY YEAR(tanggal_pinjam),MONTH(tanggal_pinjam)
                    ORDER BY YEAR(tanggal_pinjam),MONTH(tanggal_pinjam)""").show()

+-----+-----+---------+--------------+
|tahun|Bulan|   Jumlah|Belum_Terbayar|
+-----+-----+---------+--------------+
| 2015|    7|7652000.0|     1888000.0|
| 2015|    8| 7.7108E7|      6.1124E7|
| 2015|    9|  1.478E8|     1.23492E8|
| 2015|   10|    1.8E7|     1620000.0|
| 2015|   11| 1.2964E7|      964000.0|
| 2015|   12|6172000.0|      652000.0|
| 2016|    1|   1.14E7|     5632000.0|
| 2016|    2| 1.7496E7|     3136000.0|
| 2016|    3| 2.4512E7|     3984000.0|
| 2016|    4| 2.1072E7|     3484000.0|
| 2016|    5| 1.4676E7|     1772000.0|
| 2016|    6|9532000.0|      428000.0|
+-----+-----+---------+--------------+



In [ ]:
#query 3
cf_buku.registerTempTable('cf_buku')
sqlContext.sql("""WITH cte AS (
                    SELECT Jumlah_Sekarang,bulan,tahun,
                    SUM(Jumlah_Sekarang) OVER (ORDER BY tahun,bulan) AS Total_Sekarang 
                    FROM (
                        SELECT COUNT(unique_id) AS Jumlah_Sekarang,
                        MONTH(tanggal_datang) AS bulan, YEAR(tanggal_datang) AS tahun
                        FROM cf_buku
                        WHERE YEAR(tanggal_datang)!=1
                        GROUP BY MONTH(tanggal_datang),YEAR(tanggal_datang)
                    ) AS total_per_bulan_tahun
                    GROUP BY bulan,tahun,Jumlah_Sekarang
                )
                SELECT bulan, tahun, Total_Sekarang,
                Total_Sekarang - LAG(Total_Sekarang,1) OVER (PARTITION BY bulan ORDER BY tahun) AS Kenaikan
                FROM cte WHERE tahun BETWEEN 2014 AND 2016
                GROUP BY Total_Sekarang,bulan,tahun
                ORDER BY bulan,tahun""").show()

+-----+-----+--------------+--------+
|bulan|tahun|Total_Sekarang|Kenaikan|
+-----+-----+--------------+--------+
|    1| 2014|        311406|    null|
|    1| 2015|        318600|    7194|
|    1| 2016|        325296|    6696|
|    2| 2014|        311776|    null|
|    2| 2015|        318994|    7218|
|    2| 2016|        325734|    6740|
|    3| 2014|        312710|    null|
|    3| 2015|        319704|    6994|
|    3| 2016|        326080|    6376|
|    4| 2014|        313606|    null|
|    4| 2015|        320444|    6838|
|    4| 2016|        327258|    6814|
|    5| 2014|        313958|    null|
|    5| 2015|        321010|    7052|
|    5| 2016|        327720|    6710|
|    6| 2014|        314180|    null|
|    6| 2015|        321390|    7210|
|    6| 2016|        328328|    6938|
|    7| 2014|        314674|    null|
|    7| 2015|        321834|    7160|
+-----+-----+--------------+--------+
only showing top 20 rows



In [ ]:
#query 4
cf_transaksi_jurusan_kategori.registerTempTable('cf_transaksi_jurusan_kategori')
sqlContext.sql("""SELECT judul,
                         nama_koleksi,
                         status_sekarang,
                         COUNT(unique_id) AS jumlah
                    FROM cf_transaksi_jurusan_kategori
                    WHERE nama_jurusan='Teknik Arsitektur'
                    AND tahun_ajaran BETWEEN '2013/2014' AND '2015/2016'
                    AND kelompok_kategori IN ('Geologi','Tata Kota dan Pertamanan')
                    GROUP BY judul,status_sekarang,nama_koleksi
                    ORDER BY COUNT(unique_id) DESC 
                    LIMIT 5""").show()

In [ ]:
#query 5 
cf_transaksi_jurusan_kategori.registerTempTable('cf_transaksi_jurusan_kategori')
sqlContext.sql("""SELECT tahun_ajaran,
                         semester,
                         COUNT(unique_id) AS Jumlah,
                         COUNT(unique_id) - FLOOR(AVG(COUNT(unique_id)) 
                         OVER (PARTITION BY tahun_ajaran ORDER BY tahun_ajaran)) 
                    AS Selisih_dengan_Rata_rata_tahunan
                    FROM cf_transaksi_jurusan_kategori
                    WHERE nama_jurusan='Teknik Industri' AND tahun_ajaran IN ('2013/2014','2014/2015','2015/2016')
                    GROUP BY tahun_ajaran,semester
                    ORDER BY tahun_ajaran,semester""").show()

In [ ]:
#query 6 => failed
mv_transaksi_kondisi_koleksi.registerTempTable('mv_transaksi_kondisi_koleksi')
sqlContext.sql("""SELECT kelompok_kategori, 
                         judul, 
                         kode_judul, 
                         kode_buku, 
                         COUNT(unique_id) AS Jumlah
                    FROM mv_transaksi_kondisi_koleksi
                    WHERE nama_koleksi IN ('Referensi','Laporan Kerja Praktek') 
                    AND tahun_ajaran IN ('2014/2015','2015/2016')
                    AND status_sekarang='hilang'
                    GROUP BY kelompok_kategori,judul,kode_judul,kode_buku
                    ORDER BY Jumlah DESC """).show()

In [ ]:
#query 7
#date_from=datetime.to_date(lit('2013-01-01')).cast(TimestampType())
"""cf_usulan.select('penerbit','count(unique_id)').\
                    filter('cf_usulan.tgl_usulan>="2013-01-01"').\
                    filter(cf_usulan.penerbit!='unknown').\
                    filter(cf_usulan.status=='Buku/AV Sedang Diolah').\
                    groupBy(lambda row: row.penerbit).\
                    reduceByKey(add).\
                    sort(desc('count(unique_id)')).\
                    limit(10).\
                    show()"""

cf_usulan.registerTempTable("cf_usulan")
sqlContext.sql("""SELECT penerbit,
                         COUNT(unique_id) AS Jumlah
                    FROM cf_usulan 
                    WHERE tgl_usulan>'2013-01-01' AND 
                    status='Buku/AV Sedang Diolah' AND 
                    penerbit!='unknown' 
                    GROUP BY penerbit 
                    ORDER BY Jumlah DESC
                    LIMIT 10""").show()#.explain()